<a href="https://colab.research.google.com/github/Trevin07/Machine-Learning-with-Python/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv("train-data.tsv", sep='\t', names=['label', 'message'])
test_data = pd.read_csv("valid-data.tsv", sep='\t', names=['label', 'message'])


vectorizer = tf.keras.layers.TextVectorization(max_tokens=1000, output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(train_data['message']).batch(32)
vectorizer.adapt(text_ds)

train_messages = vectorizer(np.array(train_data['message']))
test_messages = vectorizer(np.array(test_data['message']))

train_labels = np.where(train_data['label'] == 'spam', 1, 0)
test_labels = np.where(test_data['label'] == 'spam', 1, 0)


In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(10000, 16, input_length=100),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_messages, train_labels, epochs=50, validation_data=(test_messages, test_labels))

In [ ]:

def predict_message(pred_text):
    pred_text = tf.convert_to_tensor([pred_text])
    pred_text = vectorizer(pred_text)
    pred = model.predict(pred_text)[0][0]
    print(f"Predicted value (raw): {pred}")  # Debugging line
    prediction = [float(pred), 'spam' if pred > 0.5 else 'ham']
    return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:

def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
